### Armington Trade Model

This notebook illustrates how a basic Armington trade model operates. As discussed below, it's setup so that aggregate demand derived from the household side of the model, can be pass through and the pattern of trade determined.

In [60]:
include("../src/ha-trade.jl")

using MINPACK # the solver.
using MINPACK
using FiniteDifferences
using ForwardDiff

#### High-Level Description

1. Environment

The ``-environment.jl`` file contains a function ``trade_flows`` and it takes endogenous objects (prices, price index, aggregate demand) and then returns a structure ``trade_stats`` which is composed of trade flows in expenditure share form, in value, tariff revenue, world demand, and the price index.  Helper files with parameters, compute the CES price index, compute all goods prices are in the environment file as well. Two important points:

- All prices are normalized so the country in the first entry's price index is one. 

- Trade shares, flows, and revenue are expressed as a matrix with rows being the buyer dimension and columns being the seller dimension. 

2. Solution 

The ``-solution.jl`` file contains the function ``trade_equilibrium`` which takes endogenous objects (wages, aggregate demand, production factors) it then computes the pattern of trade and world wide demand for each countries commodity via the function ``trade_flows`` function and compares it against the production in each country. The ``trade_equilibrium`` function contains key word arguments for either solver (default) or grabbing the output.

---

#### Computing Trade Flows

In [33]:
trade_params()

(θ = 4.0, τ = [0.0 0.0; 0.0 0.0], d = [1.0 1.5; 1.5 1.0], A = [1.0, 1.0], Ncntry = 2)

Here ``trade_params()`` is a function from the ``Parameters.jl`` package that preloads parameter values. They are set up in the following way:

```
trade_params = @with_kw (
    θ = 4.0, # Armington Elasticity, controls elasticity of substitution in CES model.
    τ = [0.0  0.0; 0.0 0.0], # tariff rate, row = buyer, column = supplier
    d = [1.0  1.5; 1.5 1.0], # trade cost, row = buyer, column = supplier
    A = [1.0, 1.0], #TFP, # Productivity in each country
    Ncntry = length(A),
    
)
```

Then below is a basic call of how to compute the trade flows. 

In [16]:
params = trade_params()

w = [1.0 ,1.0] # the wage rate in each country

N = [1.0 ,1.0] # the wage rate in each country

p, Pindex = goods_prices(w, params )

AD = (w .* N) ./ Pindex 

# this is aggregate demand. Needs to be in local consumption units

trade = trade_flows(p, Pindex, AD, params );

**Note** this is setup a little bit differently than standard trade settings. Here I'm de-linking the notion between income, production, and demand. What this means is wages -> goods prices. Then goods prices + aggregate demand of goods of AD -> where a country satisfies it's demand and by how much. An equillibrium is some notion about how wages (and income more broadly) relate to aggregate demand. The standard trade setting would say that whatever you eat $= \frac{w_{i}*L_{i}}{P_{i}}$ in final consumption units, but here it needs not be that.

In [35]:
trade.trade_share

2×2 Matrix{Float64}:
 0.771429  0.228571
 0.228571  0.771429

This is each country's expenditure share of goods from different locations. What this says is ``0.77`` in the first row, first column means that country one buys 77 percent of stuff from it self and then the first row, second column means that country one buys 23 percent of it's stuff from country two. 

In [36]:
trade.τ_revenue

2×2 Matrix{Float64}:
 0.0  0.0
 0.0  0.0

Here are things expressed in value terms. This is setup so it's in units of the commodity the country produces.

In [37]:
trade.trade_value

2×2 Matrix{Float64}:
 0.771429  0.228571
 0.228571  0.771429

Then because everything is symmetric here it turns out to be the same as the shares.

Now this is setup so that we can match up demand with supply. How does this work. Now world demand for a country's commodity is the sum down a column.

In [38]:
trade.world_demand

2-element Vector{Float64}:
 0.9999999999999997
 0.9999999999999997

---
#### Computing a (Trade) Equilibrium

In a simple static, trade equilibrium one is looking for wage rate $w$ such that world demand = world supply for each commodity. Pretty simple. This is how it works below in a 2 country, asymmetric world.

In [61]:
@code_warntype trade_equilibrium(w, AD, N, [0.0, 0.0] , params) 

MethodInstance for trade_equilibrium(::Vector{Float64}, ::Matrix{Float64}, ::Vector{Float64}, ::Vector{Float64}, ::NamedTuple{(:θ, :τ, :d, :A, :Ncntry), Tuple{Float64, Matrix{Float64}, Matrix{Float64}, Vector{Float64}, Int64}})
  from trade_equilibrium(w, AD, N, τrev, trade_params; output) in Main at C:\github\heterogeneous-agent-trade\src\armington-trade-solution.jl:4
Arguments
  #self#::Core.Const(trade_equilibrium)
  w::Vector{Float64}
  AD::Matrix{Float64}
  N::Vector{Float64}
  τrev::Vector{Float64}
  trade_params::NamedTuple{(:θ, :τ, :d, :A, :Ncntry), Tuple{Float64, Matrix{Float64}, Matrix{Float64}, Vector{Float64}, Int64}}
Body::Vector{Float64}
1 ─ %1 = Main.:(var"#trade_equilibrium#366")("solver", #self#, w, AD, N, τrev, trade_params)::Vector{Float64}
└──      return %1



In [52]:
N = [1.0 ,1.0]

d = [1.0 2.0; 3.0 1.0] 
τ = [0.0 0.0; 0.0 0.0]

params = trade_params(d = d, τ = τ)

Ncntry = params.Ncntry

wage(x) = x[1:Ncntry]

ADemand(x) = ( x[1:Ncntry].*N .+ x[Ncntry+1:end] )./ goods_prices(x[1:Ncntry], params)[2]
    
τrevenue(x) = x[Ncntry+1:end]

f(x) = trade_equilibrium(wage(x), ADemand(x), N, τrevenue(x) , params) 

f (generic function with 1 method)

Then what I'm exploring below is also how this works with different methods to compute the jacobian of the system of equations. ``ForwardDiff`` is the autodifferenation package. 

In [53]:
function f!(fvec, x)

    fvec .= f(x)

end

function j!(jvec, x)
    
    jvec .= ForwardDiff.jacobian(f, x)

end

function g!(jvec, x)
    
    jvec .= jacobian(central_fdm(6, 1), f, x)[1]

end

g! (generic function with 1 method)

In [56]:
initial_x = [1.0 ,1.0 , 0.0, 0.0]

@time ForwardDiff.jacobian(f, initial_x)

  0.000075 seconds (73 allocations: 7.031 KiB)


4×4 Matrix{Float64}:
 -0.373716  -0.550887  -0.888889  -0.0357143
 -1.43521    0.359809  -0.111111  -0.964286
  0.0        0.0        1.0        0.0
  0.0        0.0        0.0        1.0

In [36]:
@time jacobian(central_fdm(2, 1), f, initial_x)[1]

  0.000344 seconds (2.06 k allocations: 116.578 KiB)


4×4 Matrix{Float64}:
 -0.373716  -0.550887  -0.888889  -0.0357143
 -1.43521    0.359809  -0.111111  -0.964286
  0.0        0.0        1.0        0.0
  0.0        0.0        0.0        1.0

In [38]:
initial_x = [1.0 ,1.0 , 0.0, 0.0]

n = length(initial_x)
diag_adjust = n - 1

sol = fsolve(f!, initial_x, show_trace = true, method = :hybr;
      ml=diag_adjust, mu=diag_adjust,
      diag=ones(n),
      mode= 1,
      tol=1e-5,
       )

Iter     f(x) inf-norm    Step 2-norm      Step time
------   --------------   --------------   --------------
     1     1.154387e-01     0.000000e+00         0.237000
     2     1.059735e-02     3.796220e-02         0.608000
     3     1.032742e-03     4.500734e-04         0.001000
     4     3.359724e-05     4.328248e-06         0.000000
     5     3.101237e-06     1.843287e-09         0.000000
     6     2.436241e-08     1.590422e-11         0.000000


Results of Nonlinear Solver Algorithm
 * Algorithm: Modified Powell
 * Starting Point: [1.0, 1.0, 0.0, 0.0]
 * Zero: [8.9528177e-316, 8.9528177e-316, 8.9528177e-316, 8.9528177e-316]
 * Inf-norm of residuals: 0.000000
 * Convergence: true
 * Message: algorithm estimates that the relative error between x and the solution is at most tol
 * Total time: 0.846000 seconds
 * Function Calls: 6
 * Jacobian Calls (df/dx): 1

In [40]:
initial_x = [1.0 ,1.0 , 0.0, 0.0]

n = length(initial_x)
diag_adjust = n - 1

sol = fsolve(f!, j!, initial_x, show_trace = true, method = :hybr;
      tol=1e-5,
       )

Iter     f(x) inf-norm    Step 2-norm      Step time
------   --------------   --------------   --------------
     1     1.154387e-01     0.000000e+00         0.000000
     2     1.059736e-02     3.796220e-02         0.000000
     3     1.032742e-03     4.500736e-04         0.000000
     4     3.359712e-05     4.328247e-06         0.001000
     5     3.101226e-06     1.843272e-09         0.000000
     6     2.436247e-08     1.590412e-11         0.000000


Results of Nonlinear Solver Algorithm
 * Algorithm: Modified Powell (User Jac, Expert)
 * Starting Point: [1.0, 1.0, 0.0, 0.0]
 * Zero: [1.024270000710379, 1.2166194421735648, 0.0, 0.0]
 * Inf-norm of residuals: 0.000000
 * Convergence: true
 * Message: algorithm estimates that the relative error between x and the solution is at most tol
 * Total time: 0.001000 seconds
 * Function Calls: 6
 * Jacobian Calls (df/dx): 1

In [59]:
initial_x = [1.0 ,1.0 , 0.0, 0.0]

n = length(initial_x)
diag_adjust = n - 1

sol = fsolve(f!, g!, initial_x, show_trace = true, method = :hybr;
      tol=1e-5,
       )

Iter     f(x) inf-norm    Step 2-norm      Step time
------   --------------   --------------   --------------
     1     1.154387e-01     0.000000e+00         0.000000
     2     1.059736e-02     3.796220e-02         0.000000
     3     1.032742e-03     4.500736e-04         0.001000
     4     3.359712e-05     4.328247e-06         0.000000
     5     3.101226e-06     1.843272e-09         0.000000
     6     2.436247e-08     1.590412e-11         0.000000


Results of Nonlinear Solver Algorithm
 * Algorithm: Modified Powell (User Jac, Expert)
 * Starting Point: [1.0, 1.0, 0.0, 0.0]
 * Zero: [1.024270000710379, 1.2166194421735643, 0.0, 0.0]
 * Inf-norm of residuals: 0.000000
 * Convergence: true
 * Message: algorithm estimates that the relative error between x and the solution is at most tol
 * Total time: 0.001000 seconds
 * Function Calls: 6
 * Jacobian Calls (df/dx): 1

In [46]:
w = sol.x[1:Ncntry]
    
τ_revenue = sol.x[Ncntry+1:end]

2-element Vector{Float64}:
 0.0
 0.0

In [47]:
p, Pindex = goods_prices(w, params)

trade = trade_equilibrium(w, (w.*N) ./ Pindex , N, τ_revenue, params, output = "all")

trade_stats{Float64}([0.9531715425424682 0.07109845816791079; 0.07109845819899127 1.145520983974573], [0.9305862144565391 0.06941378554346089; 0.05843935723398401 0.941560642766016], [0.0 0.0; 0.0 0.0], [1.0242700007414594, 1.2166194421424839], [1.0; 1.1924427204563635;;])

In [48]:
trade.trade_share

2×2 Matrix{Float64}:
 0.930586   0.0694138
 0.0584394  0.941561

So in this setting country one is buys 93 percent from itself, 7 percent from country 2. The assymetry here arises because the country 2 faces a larger trade cost to buy from one than vice versa.

As one check of this, we can compare the real wage vs. the ACR formula... the trade share to the power $1/(1-\theta)$.

In [49]:
(trade.trade_share[1,1])^(1.0 / (1.0 - params.θ))

1.0242700211949352

In [50]:
w[1]

1.024270000710379

So what is happening here is that the real wage is corresponding with the ACR formula. And then below is the real wage for the second country.

In [51]:
w[2] / Pindex[2]

1.0202749543457719

In [52]:
(trade.trade_share[2,2])^(1.0 / (1.0 - params.θ))

1.0202749747504303

In [53]:
trade.world_demand

2-element Vector{Float64}:
 1.0242700007414594
 1.2166194421424839

And then to match this up with production, we price the value of production at the price $p_{ii}$ so

In [54]:
p[1,1] * params.A[1] * N[1]

1.0242700211949352

In [55]:
p[2,2] * params.A[2] * N[2]

1.2166194665049506

So what we see is the value of stuff produced is matching up with the value of stuff produced. Also, note that when expressed in value terms, it factors in to the equation that some of the goods "melt" away because of the trade costs or go towards tariffs.